# NER on Reddit Data

In this example we work through building a NER tool for extracting organization mentions from Reddit data to create a frequency table.

In [1]:
import spacy
from spacy import displacy
import pandas as pd
from collections import Counter

In [2]:
nlp = spacy.load('en_core_web_sm')

The `get_orgs` function will consume a length of text, and return a list of mentioned organizations:

In [3]:
BLACKLIST = ['ev', 'covid', 'etf', 'nyse', 'sec', 'spac', 'fda',
             'fed', 'treasury', 'eu', 'cnbc', 'faq', 'company']

def get_orgs(text):
    # process the text with our SpaCy model to get named entities
    doc = nlp(text)
    # initialize list to store identified organizations
    org_list = []
    for entity in doc.ents:
        # here we modify the original code to check that entity text is not equal to one of our 'blacklisted' organizations
        # (we also add .lower() to lowercase the text, this allows us to match both 'nyse' and 'NYSE' with just 'nyse')
        if entity.label_ == 'ORG' and entity.text.lower() not in BLACKLIST:
            org_list.append(entity.text)
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list

## Applying NER

All we need to do now is load in the */r/investing* data and apply the `get_orgs` function to our text column to create a new `organizations` column.

1. Load the data and view the top five rows with `df.head()`:

In [4]:
df = pd.read_csv('./data/reddit_investing.csv', sep='|')
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio
0,1.614290e+09,0.0,t3_lshtjn,10.0,Bloomberg article: [https://www.bloomberg.com/...,investing,Fed Views Rising Yields as Bullish Sign Reflec...,10.0,0.86
1,1.614286e+09,0.0,t3_lsgahw,56.0,Given the recent downturn in stocks especially...,investing,ARK ETFs implosion risk ------------------------,56.0,0.83
2,1.614283e+09,0.0,t3_lsf8td,1.0,[https://twitter.com/desogames/status/13649710...,investing,The Counter-Party Risk Bubble,1.0,0.53
3,1.614282e+09,0.0,t3_lsf3nh,6.0,"When you think of futures, what comes to your ...",investing,Futures were made for days like these,6.0,0.62
4,1.614278e+09,0.0,t3_lsdcib,3.0,I've been on this sub for quite some time and ...,investing,Let's talk about liquidity premiums,3.0,0.67


2. Extract mentioned organizations from `selftext` and add to a new column called `organizations`:

In [5]:
df['organizations'] = df['selftext'].apply(get_orgs)
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,organizations
0,1.614290e+09,0.0,t3_lshtjn,10.0,Bloomberg article: [https://www.bloomberg.com/...,investing,Fed Views Rising Yields as Bullish Sign Reflec...,10.0,0.86,"[Rebound \n&gt, Raphael Bostic, St. Louis Fed..."
1,1.614286e+09,0.0,t3_lsgahw,56.0,Given the recent downturn in stocks especially...,investing,ARK ETFs implosion risk ------------------------,56.0,0.83,[ARK]
2,1.614283e+09,0.0,t3_lsf8td,1.0,[https://twitter.com/desogames/status/13649710...,investing,The Counter-Party Risk Bubble,1.0,0.53,"[Citadel, OWN, ITM]"
3,1.614282e+09,0.0,t3_lsf3nh,6.0,"When you think of futures, what comes to your ...",investing,Futures were made for days like these,6.0,0.62,[NQ]
4,1.614278e+09,0.0,t3_lsdcib,3.0,I've been on this sub for quite some time and ...,investing,Let's talk about liquidity premiums,3.0,0.67,[]


*(This step can take a long time to run. It can be useful to break larger datasets into more manageable chunks if required)*

Now we have a list of all mentioned organizations contained within the `organizations` column. We will now take the full column, merge each list, and use `Counter` to create a frequency table of organization mentions.

In [6]:
# merge organizations column into one big list
orgs = df['organizations'].to_list()
orgs = [org for sublist in orgs for org in sublist]
orgs[:10]

['Rebound  \n&gt',
 'Raphael Bostic',
 'St. Louis Fed',
 'Bullard',
 'the Atlanta Fed',
 'Bostic',
 'ARK',
 'Citadel',
 'OWN',
 'ITM']

## Create Frequency Table

In [7]:
from collections import Counter

In [8]:
# create dictionary of organization mention frequency
org_freq = Counter(orgs)

We now have a *Counter* dictionary containing all of our organization labels as *keys*, and their mention frequency as *values*. The `most_common(n)` method allows us to view the **n** most frequently mentioned organizations:

In [9]:
org_freq.most_common(10)

[('Amazon', 26),
 ('Apple', 25),
 ('TSLA', 18),
 ('PE', 16),
 ('EPS', 16),
 ('ARK', 15),
 ('EBITDA', 15),
 ('GM', 14),
 ('Google', 14),
 ('Nasdaq', 12)]